In [52]:
# !pip install openai aiohttp nest_asyncio

import json
from IPython.display import Markdown, display
import textwrap
import asyncio
import openai
import aiohttp
import nest_asyncio

nest_asyncio.apply()
openai.api_key = 'lm-studio'

code_highlight = ", the code in response will be like this ```{program_language_name}"

def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

async def fetch_completion(my_msg, code_highlights = False):
    async with aiohttp.ClientSession() as session:
        async with session.post(
            'http://localhost:1234/v1/chat/completions',
            headers={
                # 'Authorization': f'Bearer {openai.api_key}',
                'Content-Type': 'application/json'
            },
            json={
                'model': 'meta-llama-3.1-8b-instruct',
                "messages": [
                  { "role": "system", "content": "if code is in response add the code like this ```{program_language_name}" },
                  { "role": "user", "content": my_msg if not code_highlights else my_msg + code_highlight }
                ],
                'temperature': 0.7,
                "max_tokens": -1, # -1 for unlimited
                "stream": True
            }
        ) as response:
            content = ""
            display_handle = display(to_markdown(content), display_id=True)
            async for line in response.content:
                line = line.decode('utf-8').strip()
                if line.startswith("data: "):
                    data = line[len("data: "):]
                    if data != '[DONE]':
                        json_data = json.loads(data)
                        delta = json_data['choices'][0]['delta']
                        if 'content' in delta:
                            content += delta['content']
                            display_handle.update(to_markdown(content))
            return content

# prompt = "Once upon a time"
# full_ans = await fetch_completion(prompt)

In [53]:
prompt = "write in max 100 words python code that get url data"
full_ans = await fetch_completion(prompt, code_highlights=True)
# print(full_ans)

> **URL Data Retrieval Using Python**
> ```python
> import requests
> 
> def get_url_data(url):
>     """
>     Retrieves data from a given URL.
> 
>     Args:
>         url (str): The URL to retrieve data from.
> 
>     Returns:
>         str: The retrieved HTML content.
>     """
>     try:
>         response = requests.get(url)
>         return response.text
>     except requests.exceptions.RequestException as e:
>         print(f"An error occurred: {e}")
>         return None
> 
> url = "https://www.example.com"
> data = get_url_data(url)
> print(data)
> ```
> This code uses the `requests` library to send a GET request to the provided URL and returns the HTML content. If an error occurs, it prints the error message and returns `None`.